<a href="https://colab.research.google.com/github/OlegKuzmin1968/Oleg_Kuzmin/blob/main/Stag_%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B8_DataSeta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стажировка

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Статистика исходных данных*

**из TG** стоп я правильно понимаю?
1. Сегментация грудной клетки
2. Сегментация плеврального выпота
3. Соответствующие исходные изображения КТ (DICOM)

это наш весь дата сет?

# Загрузка

In [3]:
from nibabel.viewers import OrthoSlicer3D
from nibabel import nifti1
import nibabel as nib
from matplotlib import pylab as plt
import matplotlib
import numpy as np
import os
from matplotlib import pyplot, cm
from PIL import Image # Синтаксис и описание функции Image.open():

# Размещение DS на GooglDiske

In [ ]:
# https://drive.google.com/drive/folders/15Qu0AyxLmbwcy7Nfka785C24GQ53jSuh
# ссылка на googl disk Дмитрия Черемисова где находится весь ДС
# https://drive.google.com/drive/folders/15Qu0AyxLmbwcy7Nfka785C24GQ53jSuh?usp=sharing

**Очень важная команда - УДАЛЕНИЕ НЕПУСТОЙ ПАПКИ В COLABE**

In [ ]:
#!rm -R /content/drive/MyDrive/Baza/Stag_1/Effusions_Directory/LUNG1-420 # УДАЛЕНИЕ НЕПУСТОЙ ПАПКИ В COLABE

**Распаковка zip файла сегментации преврального выпота**

In [ ]:
#  -q - не выводить на экран информацию о ходе процесса
#  -d - /.../ куда разместить распакованные данные
#  '/ '  - путь к файлу zip

#!unzip -q '/content/drive/MyDrive/Baza/Stag_1/Effusions_Directory/PleThora Effusions June 2020.zip' -d /content/drive/MyDrive/Baza/Stag_1/
#!unzip -q '/content/drive/MyDrive/Baza/Stag_1/Thoracic_Directory/PleThora Thoracic_Cavities June 2020.zip' -d /content/drive/MyDrive/Baza/Stag_1/

Места для хранения:

Файл zip сегментированных выпотов - /content/drive/MyDrive/Baza/Stag_1/Effusions_Directory

Файлы сегментированного выпота пациентов - /content/drive/MyDrive/Baza/Stag_1/Effusions

Файлы с np-массивами сегментированных выпотов для каждого пациента - /content/drive/MyDrive/Baza/Stag_1/Effusions_np

Файл zip оригинальных снимков - /content/drive/MyDrive/Baza/Stag_1/Original_Directory

Файлы со снимками пациентов (несколько) - /content/drive/MyDrive/Baza/Stag_1/Original_Directory/NSCLC-Radiomics/LUNG1_090

Файлы с np-массивами оригинальных снимков нескольких пациентов - /content/drive/MyDrive/Baza/Stag_1/Original_np

**Содержание zip файла сегментированых выпотов из предоставленного заказчиком Data Seta**

Pleural Effusion Segmentations (NIfTI, 1.7 MB zip)

 - PleThora Effusions June 2020.zip

    - Effusions

       - LUNG1-(001 - 420)

         - LUNG1-420_effusion_third_reviewer.nii.gz   # Загружаем этот файл

                - LUNG1-420_effusion_third_reviewer.nii




# **Список пациентов для которых сделана сегментации преврального выпота:**

In [4]:
Effusions_list = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Effusions')

print ('Список пациентов для которых сделана сегментации преврального выпота: ', type (Effusions_list))
print ('Количество пациентов в списке: ', len(Effusions_list))
print ('Сам список: ', Effusions_list)

Список пациентов для которых сделана сегментации преврального выпота:  <class 'list'>
Количество пациентов в списке:  78
Сам список:  ['LUNG1-001', 'LUNG1-002', 'LUNG1-005', 'LUNG1-008', 'LUNG1-013', 'LUNG1-016', 'LUNG1-018', 'LUNG1-024', 'LUNG1-026', 'LUNG1-028', 'LUNG1-035', 'LUNG1-038', 'LUNG1-042', 'LUNG1-046', 'LUNG1-050', 'LUNG1-051', 'LUNG1-053', 'LUNG1-060', 'LUNG1-065', 'LUNG1-068', 'LUNG1-088', 'LUNG1-091', 'LUNG1-093', 'LUNG1-095', 'LUNG1-097', 'LUNG1-104', 'LUNG1-107', 'LUNG1-113', 'LUNG1-115', 'LUNG1-117', 'LUNG1-128', 'LUNG1-133', 'LUNG1-143', 'LUNG1-148', 'LUNG1-149', 'LUNG1-156', 'LUNG1-168', 'LUNG1-170', 'LUNG1-177', 'LUNG1-186', 'LUNG1-195', 'LUNG1-196', 'LUNG1-205', 'LUNG1-229', 'LUNG1-242', 'LUNG1-249', 'LUNG1-252', 'LUNG1-253', 'LUNG1-277', 'LUNG1-285', 'LUNG1-286', 'LUNG1-291', 'LUNG1-296', 'LUNG1-299', 'LUNG1-301', 'LUNG1-303', 'LUNG1-307', 'LUNG1-312', 'LUNG1-314', 'LUNG1-320', 'LUNG1-325', 'LUNG1-331', 'LUNG1-336', 'LUNG1-340', 'LUNG1-343', 'LUNG1-348', 'LUNG1-

## **Список исследований каждого пациента, для которого сделана сегментации преврального выпота:**

In [5]:
LUNG1_001 = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Effusions/LUNG1-001')
print ('Список исследований пациентов для которых сделана сегментации преврального выпота: ', type (LUNG1_001))
print ('Количество исследований в списке: ', len(LUNG1_001))
print ('Сам список исследований: ', LUNG1_001)

Список исследований пациентов для которых сделана сегментации преврального выпота:  <class 'list'>
Количество исследований в списке:  2
Сам список исследований:  ['LUNG1-001_effusion_first_reviewer.nii.gz', 'LUNG1-001_effusion_second_reviewer.nii.gz']


# **Список пациентов c оригинальными снимками:**

## **Получим список всех имеющихся в папке Original_Directory/NSCLC-Radiomics  пациентов**

(код для моего диска в Colab) По хорошему, здесь должен быть код для чтения такой же папки по ссылке с общим доступом.)

In [6]:
# Присвоим переменной q путь к папке где хранятся все снимки
q = '/content/drive/MyDrive/Baza/Stag_1/Original_Directory/NSCLC-Radiomics'
print (' q-', q,'\n','Тип переменной q -', type(q))

 q- /content/drive/MyDrive/Baza/Stag_1/Original_Directory/NSCLC-Radiomics 
 Тип переменной q - <class 'str'>


In [7]:
# Получим список пациентов с оригинальными снимками

Original_list = os.listdir(q) # занесем всех пациентов с оригинальными снимками в список
Original_list.sort()          # отсортируем список
print ('Список всех имеющихся в папке Original_Directory/NSCLC-Radiomics пациентов: ', type (Original_list))
print ('Количество пациентов в списке: ', len(Original_list))
print ('Сам список: ', Original_list)

Список всех имеющихся в папке Original_Directory/NSCLC-Radiomics пациентов:  <class 'list'>
Количество пациентов в списке:  12
Сам список:  ['LUNG1_090', 'LUNG1_091', 'LUNG1_092', 'LUNG1_093', 'LUNG1_094', 'LUNG1_096', 'LUNG1_097', 'LUNG1_098', 'LUNG1_099', 'LUNG1_396', 'LUNG1_397', 'LUNG1_400']


## **Получим список пациентов у которых нет оригинальных снимков**

(порядковые номера этих пациентов отсутствуют в папке оригинальных снимков). Такое построение DS мне не совсем нравиться из-за того, что в дальнейшей работе используется множество различных счетчиков и ссылок и в их работе возможны некие сбои. Для того чтобы исключить подобные моменты и получать более правильную статистику, я бы не стал нарушать сквозную нумерацию пациентов. Что касается причины, подобных пропусков (058,059,060,___,062,063), то она понятна: у пациента 061 снимок оказался некачественный, и его тупо del.    

In [12]:
# Получим список пациентов у которых нет оригинальных снимков

Original_list_none = ['LUNG1_003','LUNG1_014','LUNG1_021','LUNG1_031','LUNG1_058','LUNG1_061','LUNG1_069','LUNG1_074','LUNG1_083','LUNG1_085','LUNG1_095','LUNG1_137','LUNG1_180','LUNG1_194','LUNG1_201','LUNG1_207','LUNG1_209','LUNG1_228','LUNG1_246','LUNG1_272']
print ('Количество пациентов у которых нет оригинальных снимков: ', len(Original_list_none))
print ('Пациенты без оригинальных снимков: ', Original_list_none)


Количество пациентов у которых нет оригинальных снимков:  20
Пациенты без оригинальных снимков:  ['LUNG1_003', 'LUNG1_014', 'LUNG1_021', 'LUNG1_031', 'LUNG1_058', 'LUNG1_061', 'LUNG1_069', 'LUNG1_074', 'LUNG1_083', 'LUNG1_085', 'LUNG1_095', 'LUNG1_137', 'LUNG1_180', 'LUNG1_194', 'LUNG1_201', 'LUNG1_207', 'LUNG1_209', 'LUNG1_228', 'LUNG1_246', 'LUNG1_272']


## **Список пациентов с нестандартными снимками:**

(информация от других участников проекта)


In [13]:
Original_list_nonstandart = ['LUNG1_111','LUNG1_396']
print ('Количество пациентов с нестандартными снимками: ', len(Original_list_nonstandart))
print ('Список пациентов с нестандартными снимками: ', Original_list_nonstandart)

Количество пациентов с нестандартными снимками:  2
Список пациентов с нестандартными снимками:  ['LUNG1_111', 'LUNG1_396']


## **Список пациентов у которых, предположительно, нет заболевания**

Предположение основано на факте отсутствия файла сегментированного выпота.

In [10]:
# Для получения списка здоровых пациентов сделаем следующее:
#      1. Добавим к списку пациентов с выпотом список нестандартных файлов

Exclusion_list = []
Exclusion_list.extend(Effusions_list)
Exclusion_list.extend(Original_list_none)
print ('Количество пациентов в листе исключения: ', len(Exclusion_list))
print ('Срез списка пациентов в листе исключения: ', Exclusion_list[-6:])

#      2. Удалим из общего списка список, полученный в первом пункте.

healthy_list = [item for item in Original_list if item not in Exclusion_list]
print ('Количество здоровых пациентов для формирования x_train( на моем диске): ', len(healthy_list))
print ('Список здоровых пациентов для формирования x_train: ', healthy_list [-3:])


Количество пациентов в листе исключения:  80
Срез списка пациентов в листе исключения:  ['LUNG1-396', 'LUNG1-416', 'LUNG1-418', 'LUNG1-420', 'LUNG1_111', 'LUNG1_396']
Количество здоровых пациентов для формирования x_train( на моем диске):  11
Список здоровых пациентов для формирования x_train:  ['LUNG1_099', 'LUNG1_397', 'LUNG1_400']


# Вывод по первой части

*У нас есть список ID пациентов для которых сделана сегментация выпота. Всего, пациентов с оригинальными снимками в DS было 422-а пациента. Из них файлы отсутствуют для 20-ти пациентов. Примем в качестве условия для формирования y_Train то, что у всех остальных пациентов, не вошедших в список с сегментацией выпота, болезнь не обнаружена. Таких пациентов будет, около 344.*

In [14]:
print ('Список всех имеющихся в папке Original_Directory/NSCLC-Radiomics пациентов: ', len(Original_list))
print ('Отсутствуют пациенты в списке оригинальных снимков: ', len(Original_list_none))
print ('Список пациентов для которых сделана сегментации преврального выпота: ', len(Effusions_list))

print ('Количество пациентов с нестандартными снимками: ', len(Original_list_nonstandart))
print ('Количество пациентов в листе исключения: ', len(Exclusion_list))
print ('Количество здоровых пациентов для формирования x_train( на моем диске): ', len(healthy_list))


Список всех имеющихся в папке Original_Directory/NSCLC-Radiomics пациентов:  12
Отсутствуют пациенты в списке оригинальных снимков:  20
Список пациентов для которых сделана сегментации преврального выпота:  78
Количество пациентов с нестандартными снимками:  2
Количество пациентов в листе исключения:  80
Количество здоровых пациентов для формирования x_train( на моем диске):  11


# Размещение DS на GooglDiske

In [ ]:
path = '/content/drive/MyDrive/Baza/Stag_1'
print('Путь к датасету', path)


Путь к датасету /content/drive/MyDrive/Baza/Stag_1


In [ ]:
# Установка параметров
DataSetPath = os.path.join(os.getcwd(), path +'/')   #'lungs\\PNGDatasePart10Copy','lungs') #Путь к датасету
print (DataSetPath)
Original_Directory = 'Originals'      # Папка в которой лежат оригиналы
Effusions_Directory = 'Effusions' #_Directory     # Папка в которй лежат сегментированные выпоты
Thoracic_Directory = 'Thoracic'       # Папка в которой лежат сегментированные грудные полости

OriginalPath=os.path.join(DataSetPath,Original_Directory)
EffusionsPath =os.path.join(DataSetPath,Effusions_Directory)
ThoracicPath=os.path.join(DataSetPath,Thoracic_Directory)


print(OriginalPath) # 'D:\\Лёгкое\\lungs\\Matveev\\lungs\\PNGDatasePart10Copy\\lungs\\Originals'
print(EffusionsPath)

/content/drive/MyDrive/Baza/Stag_1/
/content/drive/MyDrive/Baza/Stag_1/Originals
/content/drive/MyDrive/Baza/Stag_1/Effusions


In [ ]:

#rr = os.listdir (DataSetPath)
#print (rr)
r = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Effusions')
print (r)                      # .ipynb_checkpoints - контрольные точки

['LUNG1-001', 'LUNG1-002', 'LUNG1-005', 'LUNG1-008', 'LUNG1-013', 'LUNG1-016', 'LUNG1-018', 'LUNG1-024', 'LUNG1-026', 'LUNG1-028', 'LUNG1-035', 'LUNG1-038', 'LUNG1-042', 'LUNG1-046', 'LUNG1-050', 'LUNG1-051', 'LUNG1-053', 'LUNG1-060', 'LUNG1-065', 'LUNG1-068', 'LUNG1-088', 'LUNG1-091', 'LUNG1-093', 'LUNG1-095', 'LUNG1-097', 'LUNG1-104', 'LUNG1-107', 'LUNG1-113', 'LUNG1-115', 'LUNG1-117', 'LUNG1-128', 'LUNG1-133', 'LUNG1-143', 'LUNG1-148', 'LUNG1-149', 'LUNG1-156', 'LUNG1-168', 'LUNG1-170', 'LUNG1-177', 'LUNG1-186', 'LUNG1-195', 'LUNG1-196', 'LUNG1-205', 'LUNG1-229', 'LUNG1-242', 'LUNG1-249', 'LUNG1-252', 'LUNG1-253', 'LUNG1-277', 'LUNG1-285', 'LUNG1-286', 'LUNG1-291', 'LUNG1-296', 'LUNG1-299', 'LUNG1-301', 'LUNG1-303', 'LUNG1-307', 'LUNG1-312', 'LUNG1-314', 'LUNG1-320', 'LUNG1-325', 'LUNG1-331', 'LUNG1-336', 'LUNG1-340', 'LUNG1-343', 'LUNG1-348', 'LUNG1-361', 'LUNG1-362', 'LUNG1-366', 'LUNG1-367', 'LUNG1-376', 'LUNG1-377', 'LUNG1-381', 'LUNG1-387', 'LUNG1-396', 'LUNG1-416', 'LUNG1-418'

In [ ]:
#Возвращает список полных путей к файлам с оригинальными изображениями
def get_original_full_paths(OriginalPath=OriginalPath):
  OriginalFileList=[]
  if SourceType=='PNG':
    OriginalFileList=os.listdir(OriginalPath)
    for index,FileName in enumerate(OriginalFileList):
      OriginalFileList[index] = os.path.join(OriginalPath,FileName)
  return OriginalFileList

In [ ]:
OriginalFileList=get_original_full_paths()#Получили список путей к файлам
print (f'Всего оригинальных изображений: {len(OriginalFileList)}')

# Всего оригинальных изображений: 1003